In [46]:
# !pip install pandahouse

In [69]:
import pandasql as ps
import pandahouse as ph
from hashlib import md5
import pandas as pd

In [47]:
connection = {'host': 'https://clickhouse.lab.karpov.courses/',
              'user': '',
              'password': ''}


In [27]:
def get_hash(x: str):
    return int(md5(x.encode('ascii')).hexdigest(), 16)

In [28]:
get_hash('gfdhdsdjk')

138337153163266979338946680060690773084

In [65]:
query = '''
select *
from default.installs
where Source in ('Source_9', 'Source_10', 'Source_11')
'''
df = ph.read_clickhouse(query, connection = connection)

In [66]:
df.head()

,InstallationDate,InstallCost,Platform,DeviceID,Source
0,2019-05-24,276,android,3275981320170642340,Source_9
1,2019-09-19,322,android,6429599748139962965,Source_9
2,2019-01-06,98,android,531140942139788876,Source_9
3,2019-04-19,326,android,2103557350163571622,Source_9
4,2019-03-23,222,android,1407523416992072760,Source_9


In [67]:
df\
.groupby('Platform')['DeviceID']\
.nunique()\
.reset_index()\
.sort_values(by = 'DeviceID', ascending = False)

,Platform,DeviceID
0,android,2409867
1,iOS,273086


In [68]:
query = '''
select Platform, count(distinct DeviceID) as cnt
from (select *
    from default.installs
    where Source in ('Source_9', 'Source_10', 'Source_11'))
group by Platform
order by cnt desc
'''
ph.read_clickhouse(query, connection = connection)

,Platform,cnt
0,android,2409867
1,iOS,273086


In [74]:
houses = pd.read_csv('/Users/i.pile/Downloads/train-2.csv')

In [81]:
houses.groupby('Street')['SalePrice'].mean().reset_index()

,Street,SalePrice
0,Grvl,130190.500000
1,Pave,181130.538514


In [82]:
simple_query = '''
select Street, avg(SalePrice) as SalePrice
from houses
group by Street
'''

ps.sqldf(simple_query, locals())

,Street,SalePrice
0,Grvl,130190.500000
1,Pave,181130.538514


In [84]:
q = '''
select city,
    uniqExactIf(id, host_identity_verified = 't') as verified,
    uniqExactIf(id, host_identity_verified != 't') as not_verified
    
from default.listings
group by city
'''
ph.read_clickhouse(q, connection = connection)

,city,verified,not_verified
0,,3,2
1,柏林,0,1
2,Berlín,2,0
3,.,12,0
4,Mitte,0,8
...,...,...,...
56,Berlin (Kreuzberg),1,0
57,Berlin-Rudow (Neukölln),1,0
58,"Baseler Straße 91 A,12205 Berlin",0,1
59,"Baseler Straße 91 A, 12205 Berlin",0,1


In [37]:
'1ghjafsk' > '!56217'

True

In [39]:
'Ö'>'o'

True

In [ ]:
'''
select count(distinct  InvoiceNo ) as invoices,
    count(distinct   CustomerID  ) as customers,
    invoices/customers as invoices_per_customer
from default.retail
where toDate(InvoiceDate) between '2011-02-01' and '2011-02-28'
'''

In [40]:
'''
select *
from default.retail
where  InvoiceNo not in ('542776', '542777')
limit 100
'''

True

In [ ]:
'''
select count(distinct ceil(UnitPrice))
from default.retail
where lower(Description) like '%babushka%boxes%'
'''

In [ ]:
'''
select lower(upper('fghdjkagdr'))
'''

In [ ]:
'''
select InvoiceNo, 
    count() as cnt
from default.retail
where lower(Description) like '%babushka%boxes%'
group by InvoiceNo
having cnt > 1
'''

In [ ]:
'''
select CustomerID,
    toDate(min( InvoiceDate )) min_dt,
    toDate(max( InvoiceDate )) max_dt,
    count(distinct  InvoiceNo ) as invoices,
    max_dt - min_dt as diff_days,
    invoices/diff_days as invoices_per_day,
    dateDiff('month', min_dt, max_dt) as diff_months,
    invoices/diff_months as invoices_per_month
    
from default.retail
where CustomerID in ('17850','14505')
group by CustomerID
'''

In [42]:
'grehaj'.upper().lower()

'grehaj'

In [44]:
a = '546327'

str(int(a)*100)

'54632700'

In [ ]:
'''
select purchase_date, sum(price*quantity) as my_sum
from project_variant_2.purchase
where category_id = 3
    and price between 100 and 300
group by purchase_date
having my_sum > 3000
'''

In [ ]:
'''
with price*quantity as f

select toStartOfMonth(purchase_date) as month, 
    sum(f) as total_price,
    count() as cnt,
    avg(f) as avg_price,
    stddevPop(f) as std_dev_price,
    min(f) as min_price,
    max(f) as max_price
from project_variant_2.purchase
where purchase_date between '2020-01-01' and '2020-06-30'
group by month
'''

In [ ]:
'''
with price*quantity as f

select toStartOfMonth(purchase_date) as month, 
    sum(f) as total_price,
    count() as cnt,
    avg(f) as avg_price,
    stddevPop(f) as std_dev_price,
    min(f) as min_price,
    max(f) as max_price,
    quantileExact(0.1)(f) as q10_exact,
    quantileExact(0.2)(f) as q20_exact
from project_variant_2.purchase
where purchase_date between '2020-01-01' and '2020-06-30'
group by month
'''

In [ ]:
'''
with price*quantity as f

select toStartOfMonth(purchase_date) as month, 
    min(f) as min_price,
    max(f) as max_price,
    argMin(purchase_id ,f) as purchase_min,
    argMax(purchase_id ,f) as purchase_max
from project_variant_2.purchase
where purchase_date between '2020-01-01' and '2020-06-30'
group by month
'''

In [ ]:
'''
select 
    toYear(birth_date) birth_year,
    toMonth(registration) reg_month,
    count()

from project_variant_2.client
group by birth_year, reg_month
'''

In [ ]:
'''
select toDayOfMonth(time) as day,
    sum(action = 'like') likes,
    sum(action = 'view') views,
    likes/views as ctr
from simulator.feed_actions
where os = 'Android'
    and toDate(time) between '2021-11-01' and '2021-11-30'
group by day
'''

In [ ]:
'''
select 
    toStartOfMonth(l.purchase_date) as month, 
    r.city as city,
    sum(price*quantity)
from project_variant_2.purchase as l
left join 
    (select distinct city_id, city
    from project_variant_2.city) as r
on l.city_id = r.city_id
group by month, city
order by month, city
'''

In [ ]:
'''
select 
    l.*
from project_variant_2.purchase as l
inner join 
    (select *
    from project_variant_2.client
    where toStartOfMonth(registration) 
        between '2019-12-01' and '2020-02-01') as r
on l.client_id = r.client_id
limit 100
'''